In [ ]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from agent.tools import simplify_explanation_tool, procedural_steps_tool, checklist_tool, generate_admin_letter_tool, summarize_document_tool
from agent.state import AgentState

def create_workflow():
    """
    Nouveau workflow LangGraph (API 2024+)
    """

    graph = StateGraph(state_schema=AgentState)
    llm = ChatOpenAI(model="gpt-4o-mini")

    def analyze_intent(state):
        question = state.input

        if any(word in question.lower() for word in ["step", "procédure", "procedure", "comment faire", "étapes"]):
            state.tool = "steps"
        else:
            state.tool = "rag"

        return state

    def generate_answer(state):
        question = state.input
        context = state.output

        prompt = f"""
        Use the following context to answer the question:

        CONTEXT:
        {context}

        QUESTION:
        {question}

        Provide a clear and helpful answer:
        """

        response = llm.invoke(prompt)
        state.final_answer = response
        return state

    graph.add_node("analyze", analyze_intent)
    graph.add_node("procedures", procedural_steps_tool)
    graph.add_node("summarize", summarize_document_tool)
    graph.add_node("generate letter", generate_admin_letter_tool)
    graph.add_node("checklist", checklist_tool)
    graph.add_node("explaination", simplify_explanation_tool)
    graph.add_node("answer", generate_answer)

    graph.set_entry_point("analyze")
    graph.add_edge("analyze", "procedures")
    graph.add_edge("analyze", "summarize")
    graph.add_edge("analyze", "generate letter")
    graph.add_edge("analyze", "checklist")
    graph.add_edge("analyze", "explaination")

    graph.add_edge("procedures", "answer")
    graph.add_edge("summarize", "answer")
    graph.add_edge("generate letter", "answer")
    graph.add_edge("checklist", "answer")
    graph.add_edge("explaination", "answer")

    graph.set_finish_point("answer")

    workflow = graph.compile()

    return display(workflow)

create_workflow()

